In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import re # 정규표현식
import urllib.request
from tqdm import tqdm # 작업 진행률 표시
from tensorflow.keras.preprocessing.text import Tokenizer # 텍스트 토큰화
from tensorflow.keras.preprocessing.sequence import pad_sequences # 길이가 다른 여러 문장의 길이를 임의로 동일하게 맞춰줌

In [3]:
data = pd.read_csv('블로그 크롤링.csv',encoding='utf-16')
data.head()

,body
0,"화이자 관련주 - 우리바이오, KPX생명과학 주가"
1,KPX생명과학 (화이자에 폐렴 항생제 중간원료 15년째 독점 공급 중) 초록색 - ...
2,바이오회사로 변신 중(feat. JW생명과학)
3,JW생명과학의 변화의 움직임이 보인다. 기존 사업영역은 수액 제조회사라면... JW...
4,내친구과학공룡 독후활동 태양계 행성 만들기!


In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

정규 표현식 - 특수문자 제거 

In [6]:
# 한글과 공백 제외 모두 제거 : 띄어쓰기 유지, 구두점 제거

data['body'] = data['body'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]',"")
data[:10]

C:\Users\User\AppData\Local\Temp/ipykernel_14936/2244394267.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['body'] = data['body'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]',"")


,body
0,화이자 관련주 우리바이오 생명과학 주가
1,생명과학 화이자에 폐렴 항생제 중간원료 년째 독점 공급 중 초록색 대한과학 백신 ...
2,바이오회사로 변신 중 생명과학
3,생명과학의 변화의 움직임이 보인다 기존 사업영역은 수액 제조회사라면 생명과학에게 암...
4,내친구과학공룡 독후활동 태양계 행성 만들기
5,내친구과학공룡 삐로링 왕자의 친구찾기 요즘 부쩍 과학 자연에 관심이 많은 두 아들 ...
6,생명과학의 바이오사이언스 인수
7,생명과학이라는 이름을 처음 들으면 뭔 바이오의약품을 만드는 게다가 이제 이런 파이...
8,아이와 과학놀이 손가락 화석 만들기
9,이번에 아이와 과학놀이로 손가락 화석 만들기 놀이를 했는데요 과학몰에서 산 사용했거...


#### 토큰화

In [17]:
# 불용어 제거 

okt = Okt()

bul_word = []
stopwords = ['의','가','이','은','들','을','하는','로','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','에서','입니다','안녕하세요']

stop_words = set(stopwords)

for word_tokens in data['body']:
    bul_word.append([word for word in okt.morphs(word_tokens) if not word in stop_words])


In [18]:
print(bul_word[:5])

[['화이자', '관련', '주', '우리', '바이오', '생명과학', '주가'], ['생명과학', '화이자', '폐렴', '항생제', '중간', '원료', '년', '째', '독점', '공급', '중', '초록색', '대', '과학', '백신', '냉장', '보관', '관련', '기업', '최근', '독감', '백신', '상온', '노출', '되었을', '때', '주가', '많이'], ['바이오', '회사', '변신', '중', '생명과학'], ['생명과학', '변화', '움직임', '보인다', '기존', '사업', '영역', '수액', '제조', '회사', '라면', '생명과학', '에게', '암', '덩어리', '제', '더블유', '바이오', '사이언스', '입양', '시킨다', '홀딩스'], ['내', '친구', '과학', '공룡', '독후', '활동', '태양계', '행성', '만들기']]


In [19]:
# 2중 리스트를 flatten 하게 만들기
final_word = sum(bul_word,[])
print(final_word)

['화이자', '관련', '주', '우리', '바이오', '생명과학', '주가', '생명과학', '화이자', '폐렴', '항생제', '중간', '원료', '년', '째', '독점', '공급', '중', '초록색', '대', '과학', '백신', '냉장', '보관', '관련', '기업', '최근', '독감', '백신', '상온', '노출', '되었을', '때', '주가', '많이', '바이오', '회사', '변신', '중', '생명과학', '생명과학', '변화', '움직임', '보인다', '기존', '사업', '영역', '수액', '제조', '회사', '라면', '생명과학', '에게', '암', '덩어리', '제', '더블유', '바이오', '사이언스', '입양', '시킨다', '홀딩스', '내', '친구', '과학', '공룡', '독후', '활동', '태양계', '행성', '만들기', '내', '친구', '과학', '공룡', '삐', '링', '왕자', '친구', '찾기', '요즘', '부쩍', '과학', '자연', '관심', '많은', '두', '아들', '덕', '정말', '번', '이상', '본', '책', '인', '내', '친구', '과학', '공룡', '삐', '링', '왕자', '친구', '찾기', '책', '생명과학', '바이오', '사이언스', '인수', '생명과학', '이라는', '이름', '처음', '들으면', '뭔', '바이오', '의약품', '만드는', '게다가', '이제', '이런', '파이프라인', '붙으면서', '생명과학', '시장', '섹시한', '회사', '아이', '과학', '놀이', '손가락', '화석', '만들기', '이번', '아이', '과학', '놀이', '손가락', '화석', '만들기', '놀이', '했는데요', '과학', '몰에서', '산', '사용', '했거든요', '인터넷', '후기', '보니', '다', '만들길', '레', '어렵지', '않은가', '보다', '하고', '가장', '안전한', '우리', '집'

In [20]:
n_list=[]
okt = Okt()
for word in final_word:
    n_list.append(okt.nouns(word))

In [21]:
final_n = sum(n_list, [])
print(final_n)

['화이자', '관련', '주', '우리', '바이오', '생명과학', '주가', '생명과학', '화이자', '폐렴', '항생제', '중간', '원료', '년', '독점', '공급', '중', '초록색', '과학', '백신', '냉장', '보관', '관련', '기업', '최근', '독감', '백신', '상온', '노출', '때', '주가', '바이오', '회사', '변신', '중', '생명과학', '생명과학', '변화', '움직임', '기존', '사업', '영역', '수액', '제조', '회사', '라면', '생명과학', '암', '덩어리', '제', '더블유', '바이오', '사이언스', '입양', '홀딩스', '내', '친구', '과학', '공룡', '독후', '활동', '태양계', '행성', '만들기', '내', '친구', '과학', '공룡', '삐', '링', '왕자', '친구', '찾기', '요즘', '부쩍', '과학', '자연', '관심', '두', '아들', '덕', '정말', '번', '이상', '책', '인', '내', '친구', '과학', '공룡', '삐', '링', '왕자', '친구', '찾기', '책', '생명과학', '바이오', '사이언스', '인수', '생명과학', '이름', '처음', '바이오', '의약품', '게다가', '이제', '파이프라인', '생명과학', '시장', '회사', '아이', '과학', '놀이', '손가락', '화석', '만들기', '이번', '아이', '과학', '놀이', '손가락', '화석', '만들기', '놀이', '과학', '산', '사용', '인터넷', '후기', '레', '가장', '우리', '집', '대한민국', '과학기술', '대전', '공연', '예술', '영화', '쇼핑', '등', '생활', '대부분', '비', '대면', '상황', '속', '과학', '예외', '올해', '크리스마스', '주간', '대한민국', '과학기술', '과학', '사이', '싱', '톡톡', '우리', '아이', '뭐'

In [22]:
korean = pd.Series(final_word).value_counts()
print("과학도서 단어 top 10")
print(korean)


과학도서 단어 top 10
과학        2949
월          274
놀이         243
실험         238
과학기술       223
          ... 
완공           1
숙소           1
느껴지지         1
외관           1
유명하답니다       1
Length: 6555, dtype: int64


In [23]:
korean.to_csv("blog_token.csv", encoding ="cp949")